# REPL.IT parser
Problem: it takes a long time to find out what tasks a student has completed. <br>
This notebook helps it. Just insert the assignments links and you'll get a table with results. <br>

Good Selenium guide - https://habr.com/post/273089/

In [108]:
#input : list of repl assignment links
links = ['https://repl.it/teacher/assignments/2299516',
         'https://repl.it/teacher/assignments/2299522',
         'https://repl.it/teacher/assignments/2299639',
         'https://repl.it/teacher/assignments/2299646']

links2 = ['https://repl.it/teacher/assignments/2299121',
         'https://repl.it/teacher/assignments/2299128',
         'https://repl.it/teacher/assignments/2299277',
         'https://repl.it/teacher/assignments/2299398']

For each link:<br>
1.Extract header - task name <br>
2.Extract all student names, which completed the task

In [117]:
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

with webdriver.Firefox() as driver:
    driver = webdriver.Firefox()
    driver.get("https://repl.it/login?goto=%2Frepls")

    #LOG IN
    driver.find_elements_by_name("username")[0].send_keys("lit1533msk@gmail.com")
    time.sleep(1)
    driver.find_elements_by_name("password")[0].send_keys("lit1533")
    time.sleep(1)
    login_button = driver.find_elements_by_xpath("/html/body/div[1]/div/div[3]/div[2]/div/div[2]/form/a")[0]
    login_button.click()
    time.sleep(5)

    #So, get the data
    df = pd.DataFrame(columns=['NAME', 'TASK'])

    for link in links2:
        print ('Current link:', link)
        driver.get(link)
        df = extract_data_from_task_page(df)


    #Process df
    df['COMPLETED'] = np.ones(len(df))
    ans = pd.pivot_table(df, values = 'COMPLETED', index = 'NAME', columns = 'TASK').fillna('-')

driver.close()

Current link: https://repl.it/teacher/assignments/2299121
Current link: https://repl.it/teacher/assignments/2299128
Current link: https://repl.it/teacher/assignments/2299277
Current link: https://repl.it/teacher/assignments/2299398


In [118]:
ans

TASK,Контрольная - В2 - №1,Контрольная - В2 - №2,Контрольная - В2 - №3,Контрольная - В2 - №4
NAME,,,,
Darina Play,1,-,-,-
Denis Shumov,-,1,1,-
Igor Korneev,-,1,-,-
Maria Mikhaylova,1,-,-,-
Nick22 Nick22,1,-,-,-
Roman Korotyshev,-,1,1,-
timofey bulgakov,1,1,-,-
Александр Левченков,1,1,1,-
Александра Козак,1,-,-,-


In [96]:
def extract_data_from_task_page(df):
    #Wait 10 sec till page is loaded   
    task = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div[2]/header/h2'))
    ).text
    
    data = driver.find_elements_by_xpath('/html/body/div/div/div[2]/div/div[2]/div[2]/div[2]')[0]

    #Iterate over data table and extract marks
    i = 1
    try:
        while i < 1000:
            name = data.find_elements_by_xpath("./a[{i}]/div[1]".format(i=i))[0].text
            result = data.find_elements_by_xpath("./a[{i}]/div[3]".format(i=i))[0].text

            if result == 'Marked Complete':
                df = df.append({"NAME": name, "TASK": task}, ignore_index = True)
            i+=1
    except IndexError:
        pass

    return df